In [1]:
import re
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd

In [2]:
data = []

my_headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

page = 810

for k in range(0, page+30, 30):
    if k == 0 :
        url = 'https://www.tripadvisor.com/Hotels-g60763-New_York_City_New_York-Hotels.html'
    else: 
        url = 'https://www.tripadvisor.com/Hotels-g60763-oa'+str(k)+'-New_York_City_New_York-Hotels.html'
    src = False
    for i in range(1,6):
        try:
            response = requests.get(url, headers = my_headers)
            src = response.content
            break
        except:
            print('failed attempt #', i)
            time.sleep(2)
    if not src:
        print('Could not get page', url)
    else:
        print('Successfully get the page', url)
    
    soup = BeautifulSoup(src.decode('ascii', 'ignore'),'lxml')
    
    divs = soup.findAll('div', {'class':re.compile('prw_rup prw_meta_hsx_responsive_listing ui_section listItem')})
    
    for div in divs:
        span = div.find('a', {'class':re.compile('ui_bubble_rating')})
        hotels = div.find('a', {'class':re.compile('property_title prominent')})
        reviews = div.find('a', {'class':re.compile('review_count')})
        if span:
            rating = span.attrs['class'][1]
        if reviews:
            hotel = hotels.text.strip()
        if reviews:
            review = reviews.text.strip()
        data.append([rating, hotel, review]) 
        
with open('Hotel Summary.txt', 'w', encoding='utf-8') as f:
    for text in data:
        f.write(text[0] + '\t' + text[1] + '\t' + text[2]+ '\n')        
    


Successfully get the page https://www.tripadvisor.com/Hotels-g60763-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa60-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa90-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa120-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa150-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa180-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa210-New_York_City_New_York-Hotels.html
Successfully get the page https://www.tripadvisor.com/Hotels-g60763-oa240-New_York_City_New_York-Hotels.html
Successfully get the page ht

In [3]:
HotelSum_cols=['rate','hotel_name','review_num']
HotelSum = pd.read_csv('Hotel Summary.txt', sep = '\t', header=None, names=HotelSum_cols)
HotelSum.head()

,rate,hotel_name,review_num
0,bubble_35,Fairfield Inn & Suites New York Manhattan/Time...,69 reviews
1,bubble_40,Crowne Plaza Times Square Manhattan,"11,740 reviews"
2,bubble_40,Park Lane Hotel,"9,123 reviews"
3,bubble_45,Arlo NoMad,"1,401 reviews"
4,bubble_40,"Martinique New York on Broadway, Curio Collect...","5,625 reviews"


In [4]:
for i in range(len(HotelSum)):
    HotelSum.iloc[i]['rate'] = HotelSum.iloc[i]['rate'][7:]
    HotelSum.iloc[i]['review_num'] = HotelSum.iloc[i]['review_num'].split()[0]
    HotelSum.iloc[i]['review_num'] = HotelSum.iloc[i]['review_num'].replace(',','')
    HotelSum.iloc[i]['review_num'] = int(HotelSum.iloc[i]['review_num'])
HotelSum.head()
    

,rate,hotel_name,review_num
0,35,Fairfield Inn & Suites New York Manhattan/Time...,69
1,40,Crowne Plaza Times Square Manhattan,11740
2,40,Park Lane Hotel,9123
3,45,Arlo NoMad,1401
4,40,"Martinique New York on Broadway, Curio Collect...",5625


In [5]:
booleans = []
for reviews in HotelSum.review_num:    
    if reviews >= 600:        
        booleans.append(True)    
    else:        
        booleans.append(False)

In [6]:
enough_reviews=pd.Series(booleans)
HotelSum[enough_reviews]
HotelSum[enough_reviews].sort_values('rate')

,rate,hotel_name,review_num
780,25,Hotel Carter,4302
474,25,Morningside Inn,936
443,30,Riverside Tower Hotel,771
471,30,Hotel Pennsylvania,1451
472,30,Days Inn by Wyndham Hotel New York City-Broadway,2294
454,30,West Side YMCA,1555
488,30,Hudson New York,8301
476,30,Midtown Convention Center Hotel,964
428,30,Royal Park Hotel,752
414,35,Hudson River Hotel,749
